In [156]:
#ライブラリをインポート
import os #OSに依存する様々な機能を利用するためのモジュール(ファイルやディレクトリ操作など)
import re #正規表現を利用するためのモジュール
import csv  #csvファイルを扱うためのモジュール
import math #数学的計算のためのモジュール
import matplotlib.pyplot as plt #グラフ描画のためのモジュール
import numpy as np  #多次元配列計算のためのモジュール
import pandas as pd #データフレームを扱うためのモジュール
from scipy.stats import kurtosis  #尖度を調べるためのモジュール
from sklearn.model_selection import train_test_split  #データをトレーニング用とテスト用に分けるためのモジュール
from sklearn import preprocessing #データを正規化するためのモジュール
from sklearn.preprocessing import StandardScaler  #データを標準化するためのモジュール
from sklearn.linear_model import LinearRegression #線型回帰
from sklearn.svm import SVC #サポートベクターマシン
from sklearn.ensemble import RandomForestClassifier #ランダムフォレスト
from sklearn.neighbors import KNeighborsClassifier  #k-近傍法
from sklearn.metrics import accuracy_score  #機械学習モデルの性能評価のためのモジュール

In [157]:
#定数を定義
BINS = 4000  #ヒストグラムのビンの数
EPSILON = .00001  #スムージングパラメータ
UPPER_LIMIT = 1.1 #静止区間の上限
LOWER_LIMIT = 0.9 #静止区間の加減
STATIONARY_INTERVALS = 5  #静止区間除去のサンプルの間隔(静止区間が何サンプル連続したら除去するか)
TRAIN_SIZE = 0.8  #ランダムフォレストのトレーニングデータの割合

In [158]:
#ディレクトリ内のデータセットのファイル名と周波数を取得する関数
def get_Hz_and_filename(path: str) -> list[int, str]:
    filename = os.listdir(path) #引数のパスのディレクトリの中のファイル名一覧を取得
    Hz_and_filename=[]  #ファイル名と周波数を格納するリストを宣言

    for file in filename:
        Hz = re.search(r'\d+', file)    #正規表現を用いてファイル名の中で一番最初に出てくる数字(周波数)を取得
        if Hz:  #数字の入っていないファイル名があるとエラーを吐くので、このif文でチェックする
            Hz_and_filename.append([int(Hz.group(0)), file])    #ファイル名と周波数を格納

    return Hz_and_filename

In [159]:
#ファイル名と周波数を分けて出力する関数
def divide_Hz_and_filename(Hz_and_filename: list[int, str]) -> tuple[list[int], list[str]]:
    Hz = []
    filename = []
    for row in Hz_and_filename:
      Hz.append(row[0])
      filename.append(row[1])

    return Hz, filename

In [160]:
#加速度データのCSVファイルから3軸加速度を取得する関数
def get_acceleration(filename: str) -> tuple[list[float], list[float], list[float]]:
    AccX, AccY, AccZ = [], [], []
    with open(filename) as f:
        reader = csv.reader(f)
        for row in reader:
            AccX.append(float(row[2]))
            AccY.append(float(row[3]))
            AccZ.append(float(row[4]))

    return AccX, AccY, AccZ

In [161]:
#静止区間を除去する関数
def remove_stationary_intervals(AccX: list[float], AccY: list[float], AccZ: list[float]) -> list[float]:
    #各軸の加速度の平均を求める
    AvgAccX = sum(AccX) / len(AccX)
    AvgAccY = sum(AccY) / len(AccY)
    AvgAccZ = sum(AccZ) / len(AccZ)

    AvgResultantAcc = math.sqrt(AvgAccX ** 2 + AvgAccY ** 2 + AvgAccZ ** 2) #重力加速度の推定値=合成加速度の平均を求める

    ResultantAcc = [math.sqrt(x ** 2 + y ** 2 + z ** 2) for x, y, z in zip(AccX, AccY, AccZ)]   #各時刻の合成加速度を求める

    #各時刻の合成加速度から静止区間(重力加速度の推定値に近い値が一定以上以上連続している区間)を除去する
    i = 0 #ループ変数
    counter = 0 #静止区間がSTATIONARY_INTERVALS分続いているかをカウントする変数
    while i < len(ResultantAcc):
        if AvgResultantAcc * LOWER_LIMIT < ResultantAcc[i] < AvgResultantAcc * UPPER_LIMIT:   #平均のLOWER_LIMIT倍~UPPER_LIMIT倍の範囲を調べる
            counter += 1    #範囲内ならカウントを増やす
            if counter == STATIONARY_INTERVALS: #カウントがSTATIONARY_INTERVALSに達したらその区間を削除
                del ResultantAcc[i+1-STATIONARY_INTERVALS:i+1]    #スライスでは選択範囲の開始位置startと終了位置stopを[start:stop]のように書くとstart <= x < stopの範囲が選択される #start番目の値は含まれるがstop番目の値は含まれない
                counter = 0 #カウンターをリセット
                i -= STATIONARY_INTERVALS   #削除した分インデックスがズレるので補正する
        else:
            counter = 0 #カウンターをリセット
        i += 1

    return ResultantAcc  #静止区間を除去した後のリストを返す

In [162]:
#連続する2サンプルの差分を取る関数
def calculate_differences_of_acceleration(ResultantAcc: list[float]) -> list[float]:
    DifferenceAcc = [math.fabs(ResultantAcc[i + 1] * 100000 - ResultantAcc[i] * 100000) for i in range(len(ResultantAcc) - 1)]  #100000倍して誤差を取る
    return DifferenceAcc

In [163]:
#KLダイバージェンス関数 #引数として与える2つの分布は非負の値の集合でなければならないことに注意
def KL_divergence(a: list[float], b: list[float]) -> float:
    min_value = min(min(a), min(b)) #a,bの最小値の小さい方
    max_value = max(max(a), max(b)) #a,bの最大値の大きい方

    #a,bのヒストグラムを作成し、同じ数のビンで区切る
    a_hist, _ = np.histogram(a, bins=BINS, range=(min_value, max_value))
    b_hist, _ = np.histogram(b, bins=BINS, range=(min_value, max_value))

    #正規化する(確率分布に変換する、合計を1にする)ために全合計で割る
    a_hist = (a_hist + EPSILON) / a_hist.sum()
    b_hist = (b_hist + EPSILON) / b_hist.sum()

    #KLダイバージェンスの値を返す
    return np.sum([ai * np.log(ai / bi) for ai, bi in zip(a_hist, b_hist)])

In [164]:
#JSダイバージェンス関数 #引数として与える2つの分布は非負の値の集合でなければならないことに注意
def JS_divergence(a: list[float], b: list[float]) -> float:
    min_value = min(min(a), min(b)) #a,bの最小値の小さい方
    max_value = max(max(a), max(b)) #a,bの最大値の大きい方

    #a,bのヒストグラムを作成し、同じ数のビンで区切る
    a_hist, _ = np.histogram(a, bins=BINS, range=(min_value, max_value))
    b_hist, _ = np.histogram(b, bins=BINS, range=(min_value, max_value))

    #正規化する(確率分布に変換する、合計を1にする)ために全合計で割る
    a_hist = (a_hist + EPSILON) / a_hist.sum()
    b_hist = (b_hist + EPSILON) / b_hist.sum()

    #2つの分布の平均値を求める
    mean_hist = (a_hist + b_hist) / 2.0

    #平均とそれぞれの分布のKLダイバージェンスを算出
    kl_a = np.sum([ai * np.log(ai / bi) for ai, bi in zip(a_hist, mean_hist)])
    kl_b = np.sum([ai * np.log(ai / bi) for ai, bi in zip(b_hist, mean_hist)])

    #JSダイバージェンスの値を返す
    return (kl_a + kl_b) / 2.0

In [165]:
#データフレームの各行の中で2番目に小さい値が格納されている場所を調べる関数(最小値は同じ確率分布同士の0.0)
def get_index_and_columns_of_second_smallest(df: pd.DataFrame) -> list[str, str]:
    index_and_columns_of_second_smallest = []  #データフレームの中で2番目に小さい値が格納されている場所のインデックス名とカラム名を格納する変数
    for i in range(len(df)):
        sorted_row = df.iloc[i].sort_values()   #.ilocでデータフレームの要素を行、列の番号の添字で指定する    #各行の要素を昇順に並び替える
        second_smallest_columns = sorted_row.index[1] #各行の2番目に小さい値が格納されているカラム[1]の名前を取得
        #second_smallest_label = df.columns.get_loc(second_smallest_index)
        index_and_columns_of_second_smallest.append((df.index[i], second_smallest_columns))    #インデックスとカラムのラベル名の組を二次元配列に追加
    return index_and_columns_of_second_smallest

In [166]:
#推定精度を算出する関数
def calculate_accuracy(index_and_columns_of_second_smallest: list[str, str]) -> tuple[float, list[int]]:
    counter = 0
    error_index_list = []
    for i in range(len(index_and_columns_of_second_smallest)):
        #インデックスとカラムのラベル名が同じならばカウンターを1増やす
        if index_and_columns_of_second_smallest[i][0] == index_and_columns_of_second_smallest[i][1]:
            counter += 1
        else:
            error_index_list.append(i)
            print(f"間違ってるやつは{i}番目の{index_and_columns_of_second_smallest[i][0]}と{index_and_columns_of_second_smallest[i][1]}です")

    return (counter / len(index_and_columns_of_second_smallest)) * 100, error_index_list  #精度を100分率で返す

In [167]:
#入力された加速度の差分のリストからヒストグラムを作る関数（ビンの範囲が全加速度の差分データの最小値〜最大値）
def create_histogram(DifferenceAcc_list: list[float]) -> np.histogram:
    min_value = min(map(lambda x:max(x), DifferenceAcc_list))   #入力されたリストの中で最も小さい数
    max_value = max(map(lambda x:max(x), DifferenceAcc_list))   #入力されたリストの中で最も大きい数

    DifferenceAcc_hist = np.zeros((len(DifferenceAcc_list), BINS), dtype=float)

    for i in range(len(DifferenceAcc_list)):
        DifferenceAcc_hist[i], _ = np.histogram(DifferenceAcc_list[i], bins=BINS, range=(min_value, max_value)) #ヒストグラムを作成し、同じ数のビンで区切る
    return DifferenceAcc_hist

In [168]:
#入力された加速度の差分のリストからヒストグラムを作る関数（ビンの範囲が各加速度の差分データの最小値〜最大値）
def create_histogram2(DifferenceAcc_list: list[float]) -> np.histogram:
    DifferenceAcc_hist = np.zeros((len(DifferenceAcc_list), BINS), dtype=float)
    for i in range(len(DifferenceAcc_list)):
        min_value = min(DifferenceAcc_list[i])
        max_value = max(DifferenceAcc_list[i])
        DifferenceAcc_hist[i], _ = np.histogram(DifferenceAcc_list[i], bins=BINS, range=(min_value, max_value)) #ヒストグラムを作成し、同じ数のビンで区切る
    return DifferenceAcc_hist

In [169]:
#KLダイバージェンスとJSダイバージェンス算出の一連の流れを自動化した関数
def KL_and_JS(path: str):
    Hz_and_filename = get_Hz_and_filename(path)
    Hz_and_filename.sort(reverse=True)  #周波数の大きい順にソート
    Hz, filename = divide_Hz_and_filename(Hz_and_filename)
    Hz = [str(hz) + "Hz" for hz in Hz]  #周波数の値+"Hz"のリストを作りデータフレームのラベルに用いる

    #使う変数を宣言
    AccX, AccY, AccZ = [], [], []
    ResultantAcc = []
    DifferenceAcc_list = []
    resultKLD = [[0.0 for j in range(len(filename))] for i in range(len(filename))]  # resultKLDの要素を0.0で初期化
    resultJSD = [[0.0 for j in range(len(filename))] for i in range(len(filename))]  # resultKLDの要素を0.0で初期化
    error_index_list = []

    #各データセットからデータを読み込み静止区間を除去したものを二次元配列に格納
    for i in filename:
        AccX, AccY, AccZ = get_acceleration(path+i)
        ResultantAcc = remove_stationary_intervals(AccX, AccY, AccZ)
        DifferenceAcc_list.append(calculate_differences_of_acceleration(ResultantAcc))

    #KLダイバージェンスの値を格納
    for i in range(len(filename)):
        for j in range(len(filename)):
            resultKLD[i][j] = KL_divergence(DifferenceAcc_list[i], DifferenceAcc_list[j])

    #JSダイバージェンスの値を格納
    for i in range(len(filename)):
        for j in range(len(filename)):
            resultJSD[i][j] = JS_divergence(DifferenceAcc_list[i], DifferenceAcc_list[j])

    #結果を出力
    df_KLD = pd.DataFrame(resultKLD, index=Hz, columns=Hz)
    display(df_KLD)
    accuracyKLD, error_index_list = calculate_accuracy(get_index_and_columns_of_second_smallest(df_KLD))
    for i in range(len(error_index_list)):
        print(filename[error_index_list[i]])
    print(f"KLダイバージェンスによる推定精度は{accuracyKLD}%です")

    df_JSD = pd.DataFrame(resultJSD, index=Hz, columns=Hz)
    display(df_JSD)
    accuracyJSD, error_index_list = calculate_accuracy(get_index_and_columns_of_second_smallest(df_JSD))
    for i in range(len(error_index_list)):
        print(filename[error_index_list[i]])
    print(f"JSダイバージェンスによる推定精度は{accuracyJSD}%です")

In [170]:
#ランダムフォレストによる機械学習モデル構築と性能評価までを自動化した関数
def random_forest(path: str):
    Hz_and_filename = get_Hz_and_filename(path)
    Hz_and_filename.sort(reverse=True)  #周波数の大きい順にソート
    Hz, filename = divide_Hz_and_filename(Hz_and_filename)

    #使う変数を宣言
    AccX, AccY, AccZ = [], [], []
    ResultantAcc = []
    DifferenceAcc_list = []

    #各データセットからデータを読み込み静止区間を除去したものを二次元配列に格納
    for i in filename:
        AccX, AccY, AccZ = get_acceleration(path+i)
        ResultantAcc = remove_stationary_intervals(AccX, AccY, AccZ)
        DifferenceAcc_list.append(calculate_differences_of_acceleration(ResultantAcc))

    DifferenceAcc_hist = create_histogram2(DifferenceAcc_list)

    x_train, x_test, y_train, y_test = train_test_split(DifferenceAcc_hist, Hz, train_size = TRAIN_SIZE, shuffle = True)

    # 学習する
    clf = RandomForestClassifier(random_state=1234)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print("正解率 = ", accuracy_score(y_test, y_pred))

In [171]:
#ランダムフォレストによる機械学習モデル構築と性能評価までを自動化した関数
def k_neighbors(path: str):
    Hz_and_filename = get_Hz_and_filename(path)
    Hz_and_filename.sort(reverse=True)  #周波数の大きい順にソート
    Hz, filename = divide_Hz_and_filename(Hz_and_filename)

    #使う変数を宣言
    AccX, AccY, AccZ = [], [], []
    ResultantAcc = []
    DifferenceAcc_list = []

    #各データセットからデータを読み込み静止区間を除去したものを二次元配列に格納
    for i in filename:
        AccX, AccY, AccZ = get_acceleration(path+i)
        ResultantAcc = remove_stationary_intervals(AccX, AccY, AccZ)
        DifferenceAcc_list.append(calculate_differences_of_acceleration(ResultantAcc))

    DifferenceAcc_hist = create_histogram2(DifferenceAcc_list)
    scaler = StandardScaler()
    scaler.fit(DifferenceAcc_hist)
    newdata = scaler.transform(DifferenceAcc_hist)

    x_train, x_test, y_train, y_test = train_test_split(newdata, Hz, train_size = TRAIN_SIZE, shuffle = True)

    # 学習する
    clf = KNeighborsClassifier()
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print("正解率 = ", accuracy_score(y_test, y_pred))

In [172]:
path = "my_walk_data/"

In [173]:
KL_and_JS(path)

,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,...,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz
100Hz,0.000000,0.106704,0.125006,0.112222,0.128883,0.117511,0.214473,0.541020,0.110465,0.022409,...,0.261273,0.939480,0.195611,0.162266,1.158896,1.036080,1.078804,1.399452,0.429284,0.242432
100Hz,0.433574,0.000000,0.105890,0.037099,0.140596,0.048078,0.379157,0.934225,0.535799,0.442808,...,0.317536,0.637187,0.448739,0.187362,0.835197,0.699717,0.699658,0.995134,0.251793,0.186527
100Hz,0.329490,0.142476,0.000000,0.149703,0.040383,0.179336,0.159417,0.445712,0.253455,0.355222,...,0.494141,1.036792,0.207167,0.161950,1.264910,1.086191,0.983091,1.467560,0.420375,0.204424
100Hz,0.675337,0.238984,0.252576,0.000000,0.279831,0.101499,0.637612,1.220173,0.751915,0.676784,...,0.485458,0.787032,0.658249,0.426561,1.035662,0.842800,0.931305,1.100209,0.475882,0.458733
100Hz,0.372782,0.178523,0.037615,0.157241,0.000000,0.189593,0.176998,0.473445,0.285519,0.397539,...,0.557636,1.094216,0.260185,0.212865,1.345868,1.149535,1.045169,1.542280,0.479027,0.249697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10Hz,4.016863,0.973679,1.217692,0.796066,1.426426,0.858785,2.672593,4.634322,4.578507,4.115627,...,2.803191,0.912086,3.559434,2.204761,1.308713,0.000000,0.719929,0.484277,1.164998,1.765592
10Hz,4.213565,0.899664,1.244697,0.879459,1.389038,0.933331,2.501342,4.699621,4.839308,4.404938,...,2.741365,0.544981,3.632303,2.035297,0.885503,0.633886,0.000000,0.405980,0.765697,1.495695
10Hz,4.990026,1.413151,1.566096,1.069585,1.835659,1.157468,3.291200,5.580835,5.289519,5.103979,...,3.041390,0.867771,4.331937,2.840393,1.335390,0.858890,0.892788,0.000000,1.535844,2.292201
10Hz,1.637925,0.279983,0.471333,0.274493,0.550437,0.299606,1.050772,2.180996,1.777267,1.693597,...,0.751672,0.367540,1.439908,0.615709,0.514083,0.414456,0.352796,0.567198,0.000000,0.496453


間違ってるやつは6番目の100Hzと50Hzです
間違ってるやつは12番目の100Hzと90Hzです
間違ってるやつは16番目の90Hzと100Hzです
間違ってるやつは17番目の90Hzと80Hzです
間違ってるやつは18番目の80Hzと100Hzです
間違ってるやつは19番目の80Hzと90Hzです
間違ってるやつは20番目の70Hzと90Hzです
間違ってるやつは21番目の70Hzと90Hzです
間違ってるやつは22番目の60Hzと50Hzです
間違ってるやつは27番目の50Hzと100Hzです
間違ってるやつは28番目の50Hzと100Hzです
間違ってるやつは29番目の50Hzと100Hzです
間違ってるやつは30番目の50Hzと100Hzです
間違ってるやつは31番目の50Hzと100Hzです
間違ってるやつは33番目の50Hzと100Hzです
間違ってるやつは34番目の50Hzと100Hzです
間違ってるやつは35番目の50Hzと80Hzです
間違ってるやつは39番目の40Hzと60Hzです
間違ってるやつは40番目の40Hzと50Hzです
間違ってるやつは41番目の30Hzと40Hzです
間違ってるやつは42番目の30Hzと20Hzです
間違ってるやつは43番目の20Hzと30Hzです
間違ってるやつは44番目の20Hzと10Hzです
間違ってるやつは47番目の10Hzと50Hzです
間違ってるやつは49番目の10Hzと50Hzです
間違ってるやつは51番目の10Hzと50Hzです
間違ってるやつは52番目の10Hzと100Hzです
間違ってるやつは53番目の10Hzと50Hzです
間違ってるやつは55番目の10Hzと100Hzです
間違ってるやつは56番目の10Hzと50Hzです
間違ってるやつは62番目の10Hzと50Hzです
walk100Hz-20230310-173457927.csv
walk100Hz-20230301-203316681.csv
walk90Hz-20191004-065122936.csv
walk90Hz-20191004-043737805.csv
walk80Hz-20191004-065533245.csv
walk80Hz-20191004-044153012.csv
walk70Hz-20191004-0

,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,...,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz
100Hz,0.000000,0.031720,0.033639,0.035260,0.034392,0.036278,0.052753,0.120715,0.024090,0.002344,...,0.071785,0.249987,0.047629,0.044166,0.291390,0.270577,0.278721,0.337129,0.123164,0.065432
100Hz,0.031720,0.000000,0.024505,0.010670,0.029467,0.012986,0.065230,0.137071,0.049014,0.029318,...,0.035762,0.148329,0.047565,0.016431,0.179965,0.163373,0.165275,0.219601,0.049949,0.024653
100Hz,0.033639,0.024505,0.000000,0.033888,0.003497,0.040644,0.015221,0.057199,0.014904,0.036156,...,0.090525,0.222823,0.012393,0.017025,0.261918,0.237122,0.227144,0.296808,0.093154,0.028360
100Hz,0.035260,0.010670,0.033888,0.000000,0.036565,0.009214,0.080140,0.157998,0.058525,0.032646,...,0.039584,0.149140,0.062467,0.028590,0.188286,0.164055,0.175936,0.216866,0.060722,0.042236
100Hz,0.034392,0.029467,0.003497,0.036565,0.000000,0.043653,0.014419,0.055275,0.013882,0.037279,...,0.099811,0.234241,0.015487,0.022891,0.275634,0.249624,0.238348,0.310077,0.103602,0.034555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10Hz,0.270577,0.163373,0.237122,0.164055,0.249624,0.165289,0.316900,0.394835,0.310456,0.261853,...,0.155958,0.077742,0.274923,0.189410,0.086646,0.000000,0.070748,0.066018,0.088549,0.171616
10Hz,0.278721,0.165275,0.227144,0.175936,0.238348,0.179644,0.294818,0.357550,0.299747,0.272265,...,0.159585,0.064090,0.254868,0.179096,0.064940,0.070748,0.000000,0.068470,0.076042,0.156281
10Hz,0.337129,0.219601,0.296808,0.216866,0.310077,0.219514,0.379774,0.454182,0.373804,0.328002,...,0.189997,0.067474,0.337661,0.245980,0.069520,0.066018,0.068470,0.000000,0.123993,0.225916
10Hz,0.123164,0.049949,0.093154,0.060722,0.103602,0.063366,0.148954,0.218312,0.141858,0.118394,...,0.053183,0.077935,0.117039,0.058626,0.087301,0.088549,0.076042,0.123993,0.000000,0.052259


間違ってるやつは6番目の100Hzと50Hzです
間違ってるやつは12番目の100Hzと90Hzです
間違ってるやつは16番目の90Hzと80Hzです
間違ってるやつは17番目の90Hzと80Hzです
間違ってるやつは18番目の80Hzと100Hzです
間違ってるやつは19番目の80Hzと90Hzです
間違ってるやつは20番目の70Hzと90Hzです
間違ってるやつは21番目の70Hzと80Hzです
間違ってるやつは22番目の60Hzと50Hzです
間違ってるやつは27番目の50Hzと100Hzです
間違ってるやつは29番目の50Hzと100Hzです
間違ってるやつは30番目の50Hzと100Hzです
間違ってるやつは34番目の50Hzと100Hzです
間違ってるやつは35番目の50Hzと70Hzです
間違ってるやつは37番目の50Hzと90Hzです
間違ってるやつは39番目の40Hzと60Hzです
間違ってるやつは40番目の40Hzと30Hzです
間違ってるやつは41番目の30Hzと40Hzです
間違ってるやつは42番目の30Hzと20Hzです
間違ってるやつは43番目の20Hzと30Hzです
間違ってるやつは45番目の10Hzと20Hzです
間違ってるやつは47番目の10Hzと50Hzです
間違ってるやつは49番目の10Hzと50Hzです
間違ってるやつは51番目の10Hzと100Hzです
間違ってるやつは52番目の10Hzと50Hzです
間違ってるやつは53番目の10Hzと50Hzです
間違ってるやつは55番目の10Hzと50Hzです
間違ってるやつは56番目の10Hzと50Hzです
間違ってるやつは57番目の10Hzと20Hzです
walk100Hz-20230310-173457927.csv
walk100Hz-20230301-203316681.csv
walk90Hz-20191004-065122936.csv
walk90Hz-20191004-043737805.csv
walk80Hz-20191004-065533245.csv
walk80Hz-20191004-044153012.csv
walk70Hz-20191004-054529826.csv
walk70Hz-20191004-033353626.csv
walk60Hz-2

In [174]:
k_neighbors(path)

正解率 =  0.38461538461538464


In [175]:
random_forest(path)

正解率 =  0.7692307692307693


In [176]:
#個々実験用
Hz_and_filename = get_Hz_and_filename(path)
Hz_and_filename.sort(reverse=True)  #周波数の大きい順にソート
Hz, filename = divide_Hz_and_filename(Hz_and_filename)

#使う変数を宣言
AccX, AccY, AccZ = [], [], []
ResultantAcc = []
DifferenceAcc_list = []

    #各データセットからデータを読み込み静止区間を除去したものを二次元配列に格納
for i in filename:
    AccX, AccY, AccZ = get_acceleration(path+i)
    ResultantAcc = remove_stationary_intervals(AccX, AccY, AccZ)
    DifferenceAcc_list.append(calculate_differences_of_acceleration(ResultantAcc))

DifferenceAcc_hist = create_histogram2(DifferenceAcc_list)

In [177]:
kinbou = []

for i in range(100):
    scaler = StandardScaler()
    scaler.fit(DifferenceAcc_hist)
    newdata = scaler.transform(DifferenceAcc_hist)

    x_train, x_test, y_train, y_test = train_test_split(newdata, Hz, train_size = TRAIN_SIZE, shuffle = True)

    # 学習する
    clf = KNeighborsClassifier()
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    kinbou.append(accuracy_score(y_test, y_pred))

In [178]:
mink = sum(kinbou) / len(kinbou)
print(mink)

0.44615384615384585


In [179]:
#尖度
DifferenceAcc_kurtosis_list = np.zeros(len(DifferenceAcc_list))
for i in range(len(DifferenceAcc_hist)):
    DifferenceAcc_kurtosis_list[i] = kurtosis(DifferenceAcc_hist[i])

In [180]:
#分散
histogram_var = np.zeros(len(DifferenceAcc_list))
for i in range(len(DifferenceAcc_hist)):
    histogram_var[i] = np.var(DifferenceAcc_list[i])
    #histogram_var[i] = np.var(DifferenceAcc_list[i])/len(DifferenceAcc_list[i])


In [181]:
X_new = np.concatenate((DifferenceAcc_kurtosis_list.reshape(-1, 1), histogram_var.reshape(-1, 1)), axis=1)
X = np.concatenate((DifferenceAcc_hist, X_new), axis=1)

In [182]:
#尖度と分散を入れた方
x_train, x_test, y_train, y_test = train_test_split(X, Hz, train_size = TRAIN_SIZE, shuffle = True)
clf = RandomForestClassifier(random_state=1234)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print("正解率 = ", accuracy_score(y_test, y_pred))

正解率 =  0.6923076923076923


In [183]:
#尖度と分散を入れない方
x_train, x_test, y_train, y_test = train_test_split(DifferenceAcc_hist, Hz, train_size = TRAIN_SIZE, shuffle = True)
clf = RandomForestClassifier(random_state=1234)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print("正解率 = ", accuracy_score(y_test, y_pred))

正解率 =  0.6153846153846154


In [184]:
#尖度と分散のみ方
x_train, x_test, y_train, y_test = train_test_split(X_new, Hz, train_size = TRAIN_SIZE, shuffle = True)
clf = RandomForestClassifier(random_state=1234)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print("正解率 = ", accuracy_score(y_test, y_pred))

正解率 =  0.23076923076923078


In [185]:
sendotoka = []
sendonasi = []

for i in range(100):
    x_train, x_test, y_train, y_test = train_test_split(X, Hz, train_size = TRAIN_SIZE, shuffle = True)
    clf = RandomForestClassifier(random_state=1234)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    sendotoka.append(accuracy_score(y_test, y_pred))


    x_train, x_test, y_train, y_test = train_test_split(DifferenceAcc_hist, Hz, train_size = TRAIN_SIZE, shuffle = True)
    clf = RandomForestClassifier(random_state=1234)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    sendonasi.append(accuracy_score(y_test, y_pred))

In [186]:
mind = sum(sendonasi) / len(sendonasi)
print(mind)

0.6446153846153845


In [187]:
minh = sum(sendotoka) / len(sendotoka)
print(minh)

0.6607692307692307
